The below code imports a sample data, preprocesses it and develops an MLR by backward elimination

In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import dataset
dataset=pd.read_csv('50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [2]:
dataset.groupby('State').agg([len, sum])

R&D Spend             Administration             Marketing Spend  \
                 len         sum            len         sum             len   
State                                                                         
California      17.0  1099180.46           17.0  2052690.62            17.0   
Florida         16.0  1291584.26           16.0  1948302.36            16.0   
New York        17.0  1295316.06           17.0  2066239.00            17.0   

                       Profit              
                   sum    len         sum  
State                                      
California  3103195.80   17.0  1766387.98  
Florida     3957176.82   16.0  1900384.39  
New York    3490882.27   17.0  1933859.59

In [3]:
# Let's say we wish to predict Profit
X=dataset.iloc[:,:-1].values
Y=dataset.iloc[:,-1].values
dataset.dtypes

R&D Spend          float64
Administration     float64
Marketing Spend    float64
State               object
Profit             float64
dtype: object

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
R&D Spend          50 non-null float64
Administration     50 non-null float64
Marketing Spend    50 non-null float64
State              50 non-null object
Profit             50 non-null float64
dtypes: float64(4), object(1)
memory usage: 2.0+ KB


In [5]:
dataset.describe()

,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000
mean,73721.615600,121344.639600,211025.097800,112012.639200
std,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,51283.140000,0.000000,14681.400000
25%,39936.370000,103730.875000,129300.132500,90138.902500
50%,73051.080000,122699.795000,212716.240000,107978.190000
75%,101602.800000,144842.180000,299469.085000,139765.977500
max,165349.200000,182645.560000,471784.100000,192261.830000


In [6]:
dataset.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [7]:
# Missing imputation not required
# Char variable (State) encoding
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder=LabelEncoder()
X[:,3]=labelencoder.fit_transform(X[:,3])
onehotencoder=OneHotEncoder(categorical_features=[3])
X=onehotencoder.fit_transform(X).toarray()

In [8]:
X[0]

array([  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
         1.65349200e+05,   1.36897800e+05,   4.71784100e+05])

In [9]:
# Avoid dummy variable trap- i.e, leave one dummy variable after one hot encoding
X=X[:,1:]
# Split data into training and testing
from sklearn.cross_validation import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [10]:
#feature scaling isn't required in MLR. Library takes care itself
# Make multiple linear regression model
from sklearn.linear_model import LinearRegression
linearregressor=LinearRegression()
linearregressor.fit(X_train,Y_train)
# Predicting the test set result
Y_pred=linearregressor.predict(X_test)

In [11]:
from sklearn.metrics import mean_squared_error, r2_score

# The coefficients
print('Coefficients: \n', linearregressor.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(Y_test, Y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, Y_pred))


Coefficients: 
 [ -9.59284160e+02   6.99369053e+02   7.73467193e-01   3.28845975e-02
   3.66100259e-02]
Mean squared error: 83502864.03
Variance score: 0.93


In [12]:

# Building the optimal model using backward elimination
import statsmodels.formula.api as sm
#Statsmodels library doesn't account for constant term/intercept in the linear regression automatically, so you need to append column with 1 for 50 rows in data
X=np.append(arr=np.ones((50,1)).astype(int),values=X,axis=1)
# Next we specify all the indexes which are to be kept in the model. We will remove indexes one by one, so list all individually now
X_opt=X[:,[0,1,2,3,4,5]]
# Select significance level of 0.05, remove variables with P value> SL(0.05) one by one
# endog means the dependent variable and exog means the independent variables
regressor_ols=sm.OLS(endog=Y,exog=X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Fri, 31 Aug 2018   Prob (F-statistic):           1.34e-27
Time:                        18:50:27   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000      3.62e+04   6.4e+04
x1           198.7888   3371.007      0.059      0.953     -6595.030  6992.607
x2           -41.8870   3256.039     -0.013      0.990     -6604.003  6520.229
x3             0.8060      0.046     17.369      0.000         0.712     0.900
x4            -0.0270      0.052     -0.517      0.608        -0.132     0.078
x5             0.0270      0.017      1.574      0.123        -0.008     0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# In the next step remove the variable with the highest p value, if it's greater than SLS. So remove x3 with p value of 0.991
X_opt=X[:,[0,1,2,4,5]]
# Select significance level of 0.05, remove variables with P value> SL(0.05) one by one
# endog means the dependent variable and exog means the independent variables
regressor_ols=sm.OLS(endog=Y,exog=X_opt).fit()
regressor_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     17.83
Date:                Fri, 31 Aug 2018   Prob (F-statistic):           7.78e-09
Time:                        18:50:27   Log-Likelihood:                -576.91
No. Observations:                  50   AIC:                             1164.
Df Residuals:                      45   BIC:                             1173.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       1.903e+04   1.84e+04      1.033      0.307     -1.81e+04  5.61e+04
x1         -1703.7028   9337.989     -0.182      0.856     -2.05e+04  1.71e+04
x2          3875.7625   9002.603      0.431      0.669     -1.43e+04   2.2e+04
x3             0.3239      0.133      2.426      0.019         0.055     0.593
x4             0.2507      0.031      7.997      0.000         0.188     0.314
==============================================================================
Omnibus:                        5.729   Durbin-Watson:                   1.266
Prob(Omnibus):                  0.057   Jarque-Bera (JB):                5.349
Skew:                          -0.461   Prob(JB):                       0.0689
Kurtosis:                       4.311   Cond. No.                     1.34e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.34e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
# In the next step remove the variable with the highest p value, if it's greater than SLS. So remove x2 with p value of 0.703
X_opt=X[:,[0,1,4,5]]
# Select significance level of 0.05, remove variables with P value> SL(0.05) one by one
# endog means the dependent variable and exog means the independent variables
regressor_ols=sm.OLS(endog=Y,exog=X_opt).fit()
regressor_ols.summary()
# Since all the p values are less than SLS. This is our final model.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.586
Method:                 Least Squares   F-statistic:                     24.14
Date:                Fri, 31 Aug 2018   Prob (F-statistic):           1.57e-09
Time:                        18:50:28   Log-Likelihood:                -577.02
No. Observations:                  50   AIC:                             1162.
Df Residuals:                      46   BIC:                             1170.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       2.066e+04   1.79e+04      1.156      0.254     -1.53e+04  5.66e+04
x1         -3699.6258   8033.729     -0.461      0.647     -1.99e+04  1.25e+04
x2             0.3247      0.132      2.455      0.018         0.058     0.591
x3             0.2518      0.031      8.130      0.000         0.189     0.314
==============================================================================
Omnibus:                        5.809   Durbin-Watson:                   1.281
Prob(Omnibus):                  0.055   Jarque-Bera (JB):                5.344
Skew:                          -0.481   Prob(JB):                       0.0691
Kurtosis:                       4.280   Cond. No.                     1.30e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
# Backward elimination in an automated way
import statsmodels.formula.api as sm
def backwardElimination(X, sl):
    numVars = len(X[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, X).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    X = np.delete(X, j, 1)
    regressor_OLS.summary()
    return X
 
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
X_Modeled = backwardElimination(X_opt, SL)
